In [1]:
import numpy as np
import pandas as pd
import simpy

In [11]:
from callcentresim.model import (
    Experiment,
    arrivals_generator,
    operator_service,
    nurse_consultation,
    warmup_complete,
    WARM_UP_PERIOD,
    RESULTS_COLLECTION_PERIOD,
    MonitoredResource
)

In [3]:
def setup_initial_conditions(
    env: simpy.Environment, 
    args: Experiment, 
    n_in_operator_queue: int,
    n_in_nurse_queue: int
):
    """Set up initial conditions with patients already in the queues."""
    initial_id = 0
    for i in range(n_in_operator_queue):
        initial_id += 1
        # Create patients with negative IDs to distinguish them as init cond.
        env.process(operator_service(-initial_id, env, args))

    for i in range(n_in_nurse_queue):
        initial_id += 1
        env.process(nurse_consultation(-initial_id, env, args))

In [26]:
def single_run(
    experiment,
    rep=0,
    wu_period=WARM_UP_PERIOD,
    rc_period=RESULTS_COLLECTION_PERIOD,
):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/paramaters to use with model

    rep: int
        The replication number.

    wu_period: float, optional (default=WARM_UP_PERIOD)
        The initial transient period of the simulation
        Results from this period are removed from final computations.

    rc_period: float, optional (default=RESULTS_COLLECTION_PERIOD)
        The run length of the model following warm up where results are
        collected.
    """

    # results dictionary.  Each KPI is a new entry.
    run_results = {}

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()

    # create the MONITORED resources
    experiment.operators = MonitoredResource(env, experiment.n_operators)
    experiment.nurses = MonitoredResource(env, experiment.n_nurses)

    # set initial conditions
    setup_initial_conditions(env, experiment, 5, 30)
    
    # we pass the experiment to the arrivals generator
    env.process(arrivals_generator(env, experiment))

    # add warm-up period event
    env.process(warmup_complete(wu_period, env, experiment))

    # clean up resources to add in any final resource usage time
    env.process(experiment.operators.end_of_run_cleanup(wu_period + rc_period))
    env.process(experiment.nurses.end_of_run_cleanup(wu_period + rc_period))

    # run for warm-up + results collection period
    env.run(until=wu_period + rc_period)

    # end of run results: calculate mean waiting time
    run_results["01_mean_waiting_time"] = np.mean(
        experiment.results["waiting_times"]
    )

    # end of run results: calculate mean operator utilisation
    # from montiored resource stats
    run_results["02_operator_util"] = (
        experiment.operators.area_resource_busy
        / (rc_period * experiment.n_operators)
    ) * 100.0

    # mean queue lengths
    run_results["03_operator_queue_length"] = (
        experiment.operators.area_n_in_queue / rc_period
    )

    # summary results for nurse process

    # end of run results: nurse waiting time
    run_results["04_mean_nurse_waiting_time"] = np.mean(
        experiment.results["nurse_waiting_times"]
    )

    run_results["05_nurse_util"] = (
        experiment.nurses.area_resource_busy
        / (rc_period * experiment.n_nurses)
    ) * 100.0

    run_results["06_nurse_queue_length"] = (
        experiment.nurses.area_n_in_queue / rc_period
    )

    # return the results from the run of the model
    return run_results

In [31]:
default_args = Experiment()
results = single_run(default_args, wu_period=0)
results

{'01_mean_waiting_time': 3.900148610299943,
 '02_operator_util': 93.50220996145579,
 '03_operator_queue_length': 6.4820469903184925,
 '04_mean_nurse_waiting_time': 70.14348918684135,
 '05_nurse_util': 99.78834142055959,
 '06_nurse_queue_length': 48.62782528748787}